In [20]:
import kotlin.math.ceil

val input = """Time:      7  15   30
            Distance:  9  40  200"""
            
            
var times = input.substringBefore('\n').substringAfter(':').replace(" ", "").toInt()    
var distances = input.substringAfter('\n').substringAfter(':').replace(" ", "").toInt()
            
var z = listOf(Pair(times, distances))


val s = z.map { (t,d) -> 
    val a = t.toDouble()
    val b = d.toDouble() 
//    val min = kotlin.math.ceil(0.5 * (a - sqrt( a*a - 4*b  ))).toInt()
//    val max = kotlin.math.floor(0.5 * (a + sqrt( a*a - 4*b  ))).toInt()
    val min = roundUp(0.5 * (a - sqrt( a*a - 4*b  )))
    val max = roundDown(.5 * (a + sqrt( a*a - 4*b  )))
    max - min + 1 
}.reduce(Int::times) 
println(s)    
            

71503
